In [1]:
from __future__ import absolute_import, division, print_function

import argparse
import csv
import logging
import os
import random
import sys

import numpy as np
# import torch

import urllib3
import json


In [5]:
def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
	 
    requestJson = { "access_key": openapi_key, "argument": { "text": text, "analysis_code": "morp" } }
	 
    http = urllib3.PoolManager()
    response = http.request( "POST", openApiURL, headers={"Content-Type": "application/json; charset=UTF-8"}, body=json.dumps(requestJson))
    
    json_data = json.loads(response.data.decode('utf-8'))
    #print(json_data)
    json_result = json_data["result"]
    flag = False
    switch = False
    wp = ""
    
    if json_result == -1: # 결과가 없을때
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason      
    else: # 결과가 나올때
        
        json_data = json.loads(response.data.decode('utf-8')) # utf-8 화
    
        json_return_obj = json_data["return_object"] # dict 접근
        
        return_result = "" # 빈 lsit 
        json_sentence = json_return_obj["sentence"] # dict 접근 (텍스트 데이터가 있음) LIST 형태임
        #print(type(json_sentence))
        
        
        for line in json_sentence:
            data = "{}\n".format(line)
            writefirst.write(data)
           
        
        # 'sentence': [{'id': 0.0, 'reserve_str': '', 'text': '<한국 :LC> 사람들은  <지난 달 25일 :DT>에 장염을 겪었다', 'morp': ...
        for json_morp in json_sentence:
            # json_morp -> 
            # {'id': 0.0, 'reserve_str': '', 'text': '<한국 :LC> 사람들은  <지난 달 25일 :DT>에 장염을 겪었다', 'morp': [{'id': 0.0, 'lemma': '<', 'type': 'SS', 'position': 0.0, 'weight': 1.0}
            # [{'id': 0.0, 'lemma': '<', 'type': 'SS', 'position': 0.0, 'weight': 1.0}...
            # ...
            
            for morp in json_morp["morp"]:
                tag = str(morp["type"])
                string = str(morp["lemma"])
                
                if string == '凸':
                    switch = True
                    continue
                
                if flag == False:
                    if string == ".":
                        continue
                        #return_result = return_result + "\n"
                    elif string == "<":
                        flag = True
                    else:
                        return_result = return_result+string+" "+tag+ " "+"-"
                        return_result = return_result + 'nope'
                else:
                    if switch == True:
                        ner = string
                        
                        wplist = wp.split()
                        #print(wplist)
                        for idx, temp in enumerate(wplist):
                            if idx == 0:
                                return_result = return_result+ temp+" "+tag+ " "+ner+"_B"
                            else:
                                return_result = return_result+ temp+" "+tag+ " "+ner+"_I"
                            return_result = return_result + 'nope'
                        switch = False
                    else:
                        if string == " ":
                            pass
                        else:
                            wp = wp + string + " "
                            
                    if string == ">":
                        flag = False
                        wp = ""
                        continue
                    #print("$$$$$$$$$$$$$$")                 
        return return_result


In [7]:
import re
import string
apikey = "api-key"
# text = "<한국:LC> 사람들은  <지난 달 25일 :DT>에 장염을 겪었다. 오늘 아침 인사했다."

text = ""
readfile = open("before9876-10000.txt", 'r', encoding = "utf-8")
writefile = open("result9876-10000.txt", 'w')
writefirst = open("bfresult9876-10000.txt", 'w') # 형태소 분석 파일


for i in range(125):
    temp = readfile.readline()
    #temp = """<남원:LC> 1433.4m <5시간:TI> 30,000원(고급 김밥 제공)<12:23:35:DT> <9월27일:DT> <민둥산:LC> <증산초등학교:OG>-<민둥산:LC>-억새능선-<지억산:LC>-불암사-<화암약수:LC> <정선:LC> 1118.8m <5시간:TI> 25,000원(식사개인) <10월2,25일:DT> <귀떼기청봉:LC>
#"""
#     #text = temp.replace("\n", " ")
    text = temp.replace("\ufeff","")
    text = temp.replace("\ufffd","")
    #print(text)
#     symbolsToRemove = "'"
#     #text = re.sub(r'['+symbolsToRemove+']', "'", text)
#     print(temp)
    p_e = re.compile(':[A-Z]+[>]')

    m_e = p_e.search(text)
    m_e = m_e.group()
    change_ = '凸' + m_e[1:]

    result = p_e.sub(change_, text)
    #print(result)
    results = do_lang(apikey, result)
    #print("ddddd", results)
    #print(results)
#     for result in results:
    results = results.split("nope")
    for row in results:
        data = "{}\n".format(row)
        writefile.write(data)
       
writefirst.close()
writefile.close()
readfile.close()


import re
import string
apikey = "7b1c0d60-2da5-4096-9add-7eaf0401f737"
#text = ["<한국:LC> 사람들은  <지난 달 25일 :DT>에 장염을 겪었다. 오늘 아침 인사했다.", "오늘 우리가 수업들은 내용은 텐서플로우를 활용한 개발이었다. 복종순 선생님의 수업을 거의 듣지 않아서 머리에 들어오는 것이 없었다 "]


text = ""
f = open("corpus.txt", 'r', encoding = "utf-8")
for i in range(10):
    temp = f.readline()
    text = text + temp

text = text.replace("\n", " ")
text = text.replace("\ufeff","")
symbolsToRemove = "'"
text = re.sub(r'['+symbolsToRemove+']', "'", text)
text
                    

                    
#result = do_lang(apikey, text)


print(results)




read = open('beforedata.txt', 'r', encoding = 'utf-8')
data = read.readline()
print(data)
reg = r':\w'
to = r'\1뷁\2'
y =''.join(re.sub(reg, to, data))
print(y)
read.close()